In [ ]:
#| include: false

import argparse
import math
import os
from copy import deepcopy
from datetime import datetime
from functools import partial
from pathlib import Path

import numpy as np
import torch
import torch.nn as nn
import yaml
from matplotlib import pyplot as plt
from ultralytics import YOLO, __version__
from ultralytics.nn.modules import Detect, C2f, Conv, Bottleneck
from ultralytics.nn.tasks import attempt_load_one_weight
from ultralytics.engine.model import Model
from ultralytics.engine.trainer import BaseTrainer
from ultralytics.utils import LOGGER, RANK, DEFAULT_CFG_DICT, DEFAULT_CFG_KEYS
from ultralytics.utils.checks import check_yaml
from ultralytics.utils.torch_utils import initialize_weights, de_parallel

import torch_pruning as tp
from fasterai.prune.all import *
from fastai.vision.all import *
from fastcore.basics import store_attr, listify, true
from torch_pruning.pruner import function

# Compatibility helper for yaml loading (not exported in ultralytics v8.1+)
def yaml_load(file):
    """Load YAML file."""
    with open(file, 'r') as f:
        return yaml.safe_load(f)

## Helpers

In [ ]:
#| include: false

def infer_shortcut(bottleneck):
    c1 = bottleneck.cv1.conv.in_channels
    c2 = bottleneck.cv2.conv.out_channels
    return c1 == c2 and hasattr(bottleneck, 'add') and bottleneck.add


class C2f_v2(nn.Module):
    # CSP Bottleneck with 2 convolutions
    def __init__(self, c1, c2, n=1, shortcut=False, g=1, e=0.5):  # ch_in, ch_out, number, shortcut, groups, expansion
        super().__init__()
        self.c = int(c2 * e)  # hidden channels
        self.cv0 = Conv(c1, self.c, 1, 1)
        self.cv1 = Conv(c1, self.c, 1, 1)
        self.cv2 = Conv((2 + n) * self.c, c2, 1)  # optional act=FReLU(c2)
        self.m = nn.ModuleList(Bottleneck(self.c, self.c, shortcut, g, k=((3, 3), (3, 3)), e=1.0) for _ in range(n))

    def forward(self, x):
        # y = list(self.cv1(x).chunk(2, 1))
        y = [self.cv0(x), self.cv1(x)]
        y.extend(m(y[-1]) for m in self.m)
        return self.cv2(torch.cat(y, 1))


def transfer_weights(c2f, c2f_v2):
    c2f_v2.cv2 = c2f.cv2
    c2f_v2.m = c2f.m

    state_dict = c2f.state_dict()
    state_dict_v2 = c2f_v2.state_dict()

    # Transfer cv1 weights from C2f to cv0 and cv1 in C2f_v2
    old_weight = state_dict['cv1.conv.weight']
    half_channels = old_weight.shape[0] // 2
    state_dict_v2['cv0.conv.weight'] = old_weight[:half_channels]
    state_dict_v2['cv1.conv.weight'] = old_weight[half_channels:]

    # Transfer cv1 batchnorm weights and buffers from C2f to cv0 and cv1 in C2f_v2
    for bn_key in ['weight', 'bias', 'running_mean', 'running_var']:
        old_bn = state_dict[f'cv1.bn.{bn_key}']
        state_dict_v2[f'cv0.bn.{bn_key}'] = old_bn[:half_channels]
        state_dict_v2[f'cv1.bn.{bn_key}'] = old_bn[half_channels:]

    # Transfer remaining weights and buffers
    for key in state_dict:
        if not key.startswith('cv1.'):
            state_dict_v2[key] = state_dict[key]

    # Transfer all non-method attributes
    for attr_name in dir(c2f):
        attr_value = getattr(c2f, attr_name)
        if not callable(attr_value) and '_' not in attr_name:
            setattr(c2f_v2, attr_name, attr_value)

    c2f_v2.load_state_dict(state_dict_v2)


def replace_c2f_with_c2f_v2(module):
    for name, child_module in module.named_children():
        if isinstance(child_module, C2f):
            # Replace C2f with C2f_v2 while preserving its parameters
            shortcut = infer_shortcut(child_module.m[0])
            c2f_v2 = C2f_v2(child_module.cv1.conv.in_channels, child_module.cv2.conv.out_channels,
                            n=len(child_module.m), shortcut=shortcut,
                            g=child_module.m[0].cv2.conv.groups,
                            e=child_module.c / child_module.cv2.conv.out_channels)
            transfer_weights(child_module, c2f_v2)
            setattr(module, name, c2f_v2)
        else:
            replace_c2f_with_c2f_v2(child_module)


def save_model_v2(self: BaseTrainer):
    """
    Disabled half precision saving. originated from ultralytics/yolo/engine/trainer.py
    """
    ckpt = {
        'epoch': self.epoch,
        'best_fitness': self.best_fitness,
        'model': deepcopy(de_parallel(self.model)),
        'ema': deepcopy(self.ema.ema),
        'updates': self.ema.updates,
        'optimizer': self.optimizer.state_dict(),
        'train_args': vars(self.args),  # save as dict
        'date': datetime.now().isoformat(),
        'version': __version__}

    # Save last, best and delete
    torch.save(ckpt, self.last)
    if self.best_fitness == self.fitness:
        torch.save(ckpt, self.best)
    if (self.epoch > 0) and (self.save_period > 0) and (self.epoch % self.save_period == 0):
        torch.save(ckpt, self.wdir / f'epoch{self.epoch}.pt')
    del ckpt

def final_eval_v2(self: BaseTrainer):
    """
    originated from ultralytics/yolo/engine/trainer.py
    """
    for f in self.last, self.best:
        if f.exists():
            strip_optimizer_v2(f)  # strip optimizers
            if f is self.best:
                LOGGER.info(f'\nValidating {f}...')
                self.metrics = self.validator(model=f)
                self.metrics.pop('fitness', None)
                self.run_callbacks('on_fit_epoch_end')

def strip_optimizer_v2(f: str | Path = 'best.pt', s: str = '') -> None:
    """
    Disabled half precision saving. originated from ultralytics/yolo/utils/torch_utils.py
    Updated for PyTorch 2.6+: use weights_only=False for loading model checkpoints.
    """
    # PyTorch 2.6+ defaults weights_only=True, but we need to load the full model object
    x = torch.load(f, map_location=torch.device('cpu'), weights_only=False)
    args = {**DEFAULT_CFG_DICT, **x['train_args']}  # combine model args with default args, preferring model args
    if x.get('ema'):
        x['model'] = x['ema']  # replace model with ema
    for k in 'optimizer', 'ema', 'updates':  # keys
        x[k] = None
    for p in x['model'].parameters():
        p.requires_grad = False
    x['train_args'] = {k: v for k, v in args.items() if k in DEFAULT_CFG_KEYS}  # strip non-default keys
    # x['model'].args = x['train_args']
    torch.save(x, s or f)
    mb = os.path.getsize(s or f) / 1E6  # filesize
    LOGGER.info(f"Optimizer stripped from {f},{f' saved as {s},' if s else ''} {mb:.1f}MB")


def train_v2(self: YOLO, pruning=False, **kwargs):
    """
    Disabled loading new model when pruning flag is set. originated from ultralytics/yolo/engine/model.py
    Updated for ultralytics v8.1+ API.
    """

    self._check_is_pytorch_model()
    if self.session:  # Ultralytics HUB session
        if any(kwargs):
            LOGGER.warning('WARNING ⚠️ using HUB training arguments, ignoring local training arguments.')
        kwargs = self.session.train_args
    
    # Save original model path before potentially replacing overrides
    model_path = self.overrides.get('model')
    
    overrides = self.overrides.copy()
    overrides.update(kwargs)
    if kwargs.get('cfg'):
        LOGGER.info(f"cfg file passed. Overriding default params with {kwargs['cfg']}.")
        cfg_overrides = yaml_load(check_yaml(kwargs['cfg']))
        # Merge cfg with existing overrides, preserving model path
        overrides.update(cfg_overrides)
    
    # Ensure model path is preserved (required by ultralytics v8.3+)
    if model_path and not overrides.get('model'):
        overrides['model'] = model_path
        
    overrides['mode'] = 'train'
    if not overrides.get('data'):
        raise AttributeError("Dataset required but missing, i.e. pass 'data=coco128.yaml'")
    if overrides.get('resume'):
        overrides['resume'] = self.ckpt_path

    self.task = overrides.get('task') or self.task
    
    # Updated for ultralytics v8.1+: use task_map dict with 'trainer' key
    trainer_cls = self.task_map[self.task]['trainer']
    self.trainer = trainer_cls(overrides=overrides, _callbacks=self.callbacks)

    if not pruning:
        if not overrides.get('resume'):  # manually set model only if not resuming
            self.trainer.model = self.trainer.get_model(weights=self.model if self.ckpt else None, cfg=self.model.yaml)
            self.model = self.trainer.model

    else:
        # pruning mode
        self.trainer.pruning = True
        self.trainer.model = self.model

        # replace some functions to disable half precision saving
        self.trainer.save_model = save_model_v2.__get__(self.trainer)
        self.trainer.final_eval = final_eval_v2.__get__(self.trainer)

    self.trainer.hub_session = self.session  # attach optional HUB session
    self.trainer.train()
    # Update model and cfg after training
    if RANK in (-1, 0):
        self.model, _ = attempt_load_one_weight(str(self.trainer.best))
        self.overrides = self.model.args
        self.metrics = getattr(self.trainer.validator, 'metrics', None)

In [ ]:
#| include: false

def prune(args):
    # load trained yolov8 model
    model = YOLO(args.model)
    model.__setattr__("train_v2", train_v2.__get__(model))
    pruning_cfg = yaml_load(check_yaml(args.cfg))
    batch_size = pruning_cfg['batch']
    
    pruning_cfg['data'] = "coco128.yaml"
    pruning_cfg['epochs'] = 10
    pruning_cfg['verbose'] = False
    
    # Disable AMP to avoid cuDNN engine issues on newer GPUs (e.g., RTX 5090/Blackwell)
    # The C2f_v2 module creates new Conv layers that may have dtype issues with AMP
    pruning_cfg['amp'] = False
    
    model.model.train()
    replace_c2f_with_c2f_v2(model.model)
    initialize_weights(model.model)
    
    # Ensure model is on the correct device with correct dtype after weight init
    device = model.device
    model.model = model.model.to(device).float()
    
    validation_model = deepcopy(model)
    metric = validation_model.val(**pruning_cfg)
    init_map = metric.box.map
    example_inputs = torch.randn(1, 3, pruning_cfg["imgsz"], pruning_cfg["imgsz"]).to(model.device)
    
    base_macs, base_nparams = tp.utils.count_ops_and_params(model.model, example_inputs)
    print(f"Before Pruning: MACs={base_macs / 1e9: .5f} G, #Params={base_nparams / 1e6: .5f} M, mAP={init_map: .5f}")
    
    for name, param in model.model.named_parameters():
        param.requires_grad = True
        
    model.train_v2(pruning=True, **pruning_cfg)

    pruning_cfg['epochs'] = 10
    
    macs_list, nparams_list, map_list, pruned_map_list = [], [], [], []
    base_macs, base_nparams = tp.utils.count_ops_and_params(model.model, example_inputs)
    
    pruning_cfg['name'] = f"baseline_val"
    pruning_cfg['batch'] = 1
    
    
    validation_model.model.model = deepcopy(model.model.model)
    metric = validation_model.val(**pruning_cfg)
    init_map = metric.box.map
    macs_list.append(base_macs)
    nparams_list.append(100)
    map_list.append(init_map)
    pruned_map_list.append(init_map)
    print(f"Before Pruning: MACs={base_macs / 1e9: .5f} G, #Params={base_nparams / 1e6: .5f} M, mAP={init_map: .5f}")
    
    for name, param in model.model.named_parameters():
        param.requires_grad = True
    
        ignored_layers = []
        unwrapped_parameters = []
        for m in model.model.modules():
            if isinstance(m, (Detect,)):
                ignored_layers.append(m)
    
    
    print(model.model.model[0].conv)
    pruner = Pruner(model.model, args.target_prune_rate, 'local', large_final, ignored_layers=ignored_layers, iterative_steps=args.iterative_steps, schedule=args.sched)

    for i in range(args.iterative_steps):
        # Get current pruning ratio using schedule's progress API
        progress = args.sched.progress(i / args.iterative_steps)
        current_ratio = args.target_prune_rate * progress

        pruner.prune_model()
        print(f"Pruning step {i+1}: progress={progress:.3f}, ratio={current_ratio:.3f}")

        print('After Pruning')
        print('Model', model.model.model[0].conv)
        print('Pruner', pruner.model.model[0].conv)

        pruning_cfg['name'] = f"step_{i}_pre_val"
        pruning_cfg['batch'] = 1
        validation_model.model.model = deepcopy(pruner.model.model)
        metric = validation_model.val(**pruning_cfg)
        pruned_map = metric.box.map
        pruned_macs, pruned_nparams = tp.utils.count_ops_and_params(pruner.model.to(default_device()), example_inputs.to(default_device()))
        
        print('After post-pruning Validation')
        print('Model', model.model.model[0].conv)
        print('Pruner', pruner.model.model[0].conv)
        
        
        current_speed_up = float(macs_list[0]) / pruned_macs
        print(f"After pruning iter {i + 1}: MACs={pruned_macs / 1e9} G, #Params={pruned_nparams / 1e6} M, "
              f"mAP={pruned_map}, speed up={current_speed_up}")

        
        # fine-tuning
        for name, param in model.model.named_parameters():
            param.requires_grad = True
        pruning_cfg['name'] = f"step_{i}_finetune"
        pruning_cfg['batch'] = batch_size  # restore batch size
        model.model = pruner.model
        model.train_v2(pruning=True, **pruning_cfg)

        print('After fine-tuning')
        print('Model', model.model.model[0].conv)
        print('Pruner', pruner.model.model[0].conv)
        
        
        # post fine-tuning validation
        pruning_cfg['name'] = f"step_{i}_post_val"
        pruning_cfg['batch'] = 1
        validation_model = YOLO(model.trainer.best)
        validation_model.model = deepcopy(model.model)
        metric = validation_model.val( **pruning_cfg)
        current_map = metric.box.map
        print(f"After fine tuning mAP={current_map}")

        print('After post fine-tuning validation')
        print('Model', model.model.model[0].conv)
        print('Pruner', pruner.model.model[0].conv)
    

        macs_list.append(pruned_macs)
        nparams_list.append(pruned_nparams / base_nparams * 100)
        pruned_map_list.append(pruned_map)
        map_list.append(current_map)

        if init_map - current_map > args.max_map_drop:
            print("Pruning early stop")
            break


    model.export(format='onnx')

## Training

In [ ]:
class Args(argparse.Namespace):
  model = 'yolov8l.pt'
  cfg = 'default.yaml'
  iterative_steps = 10
  target_prune_rate = 0.15
  max_map_drop = 0.2
  sched = Schedule(partial(sched_onecycle,  α=10, β=4))

args=Args()
prune(args)

Ultralytics 8.3.162 🚀 Python-3.12.11 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 5090, 32109MiB)
YOLOv8l summary (fused): 121 layers, 43,668,288 parameters, 0 gradients, 165.2 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 4049.2±1625.9 MB/s, size: 51.0 KB)


val: Scanning /home/nathan/Developer/FasterAI-Labs/Projects/ALX Systems/datasets/coco128/label
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%


                   all        128        929      0.731      0.768      0.828       0.66
Speed: 0.7ms preprocess, 3.1ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to runs/detect/val8
Before Pruning: MACs= 82.72641 G, #Params= 43.69152 M, mAP= 0.66035
Ultralytics 8.3.162 🚀 Python-3.12.11 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 5090, 32109MiB)
engine/trainer: agnostic_nms=False, amp=False, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=coco128.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, 

train: Scanning /home/nathan/Developer/FasterAI-Labs/Projects/ALX Systems/datasets/coco128/lab


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 505.5±201.7 MB/s, size: 52.5 KB)


...

After fine-tuning
Model Conv2d(3, 54, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
Pruner Conv2d(3, 54, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
Ultralytics 8.3.162 🚀 Python-3.12.11 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 5090, 32109MiB)
YOLOv8l summary (fused): 121 layers, 32,416,863 parameters, 0 gradients, 123.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 5168.4±959.1 MB/s, size: 53.4 KB)


val: Scanning /home/nathan/Developer/FasterAI-Labs/Projects/ALX Systems/datasets/coco128/label
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%


                   all        128        929      0.937      0.892      0.939      0.806
Speed: 0.1ms preprocess, 6.3ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs/detect/step_9_post_val2
After fine tuning mAP=0.8059532806050649
After post fine-tuning validation
Model Conv2d(3, 54, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
Pruner Conv2d(3, 54, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
Ultralytics 8.3.162 🚀 Python-3.12.11 torch-2.9.1+cu128 CPU (Intel Core(TM) i9-14900KS)
YOLOv8l summary (fused): 121 layers, 32,416,863 parameters, 0 gradients, 123.4 GFLOPs

PyTorch: starting from 'runs/detect/step_9_finetune2/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (124.2 MB)

ONNX: starting export with onnx 1.17.0 opset 10...


W0205 17:34:38.183000 260195 site-packages/torch/onnx/_internal/exporter/_compat.py:114] Setting ONNX exporter to use operator set version 18 because the requested opset_version 10 is a lower version than we have implementations for. Automatic version conversion will be performed, which may not be successful at converting to the requested version. If version conversion is unsuccessful, the opset version of the exported model will be kept at 18. Please consider setting opset_version >=18 to leverage latest ONNX features
The model version conversion is not supported by the onnxscript version converter and fallback is enabled. The model will be converted using the onnx C API (target version: 10).
Failed to convert the model to the target version 10 using the ONNX C API. The model was not modified
Traceback (most recent call last):
  File "/home/nathan/miniconda3/envs/dev/lib/python3.12/site-packages/onnxscript/version_converter/__init__.py", line 127, in call
    converted_proto = _c_api_

Applied 1 of general pattern rewrite rules.
ONNX: slimming with onnxslim 0.1.59...
ONNX: export success ✅ 2.9s, saved as 'runs/detect/step_9_finetune2/weights/best.onnx' (123.8 MB)

Export complete (3.4s)
Results saved to /home/nathan/Developer/FasterAI-Labs/gh/fasterai/nbs/tutorials/prune/runs/detect/step_9_finetune2/weights
Predict:         yolo predict task=detect model=runs/detect/step_9_finetune2/weights/best.onnx imgsz=640  
Validate:        yolo val task=detect model=runs/detect/step_9_finetune2/weights/best.onnx imgsz=640 data=/home/nathan/miniconda3/envs/dev/lib/python3.12/site-packages/ultralytics/cfg/datasets/coco128.yaml  
Visualize:       https://netron.app


## Post-Training Checks

In [ ]:
model = YOLO('runs/detect/step_9_finetune2/weights/best.pt')

In [ ]:
example_inputs = torch.randn(1, 3, 640, 640).to(model.device)

In [ ]:
base_macs, base_nparams = tp.utils.count_ops_and_params(model.model, example_inputs); base_macs, base_nparams

(61848891200.0, 32436843)

In [ ]:
results = model.val(
                data='coco128.yaml',
                batch=1,
                imgsz=640,
                verbose=False,
            )

Ultralytics 8.3.162 🚀 Python-3.12.11 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 5090, 32109MiB)
YOLOv8l summary (fused): 121 layers, 32,416,863 parameters, 0 gradients, 123.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 5274.3±1449.8 MB/s, size: 56.4 KB)


val: Scanning /home/nathan/Developer/FasterAI-Labs/Projects/ALX Systems/datasets/coco128/label
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%


                   all        128        929      0.949       0.89       0.94      0.813
Speed: 0.1ms preprocess, 5.9ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs/detect/val10


In [ ]:
results

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 11, 13, 14, 15, 16, 17, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 67, 68, 69, 71, 72, 73, 74, 75, 76, 77, 79])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0